## Processing wound images rectifies wound borders,extracts geometric features from depth, and prepares for clustering.

In [ ]:
# --- Imports ---
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d
from scipy.stats import skew, kurtosis
from scipy.fft import fft
from scipy.integrate import simpson
from skimage.measure import find_contours
from scipy.ndimage import map_coordinates, gaussian_filter

# --- Load Input Files ---
def load_inputs(name: str, base_path: str):
    img = cv2.imread(f"{base_path}/images/{name}.png")[..., ::-1]
    depth = cv2.imread(f"{base_path}/depth_maps/{name}.png", cv2.IMREAD_ANYDEPTH)
    wound = cv2.imread(f"{base_path}/wound_masks/{name}.png", cv2.IMREAD_GRAYSCALE)
    body = cv2.imread(f"{base_path}/body_masks/{name}.png", cv2.IMREAD_GRAYSCALE)

    # Safe check: Load marker only if exists
    marker_path = f"{base_path}/marker_masks/{name}.png"
    marker = cv2.imread(marker_path, cv2.IMREAD_GRAYSCALE) if os.path.exists(marker_path) else None

    # Apply masks
    depth = cv2.bitwise_and(depth, depth, mask=body)
    if marker is not None:
        depth = cv2.bitwise_and(depth, depth, mask=~marker)
        body = cv2.bitwise_and(body, body, mask=~marker)

    return img, depth, wound, body

# --- Z-score Filtering ---
def filter_body_with_zscore(depth, body):
    z_score = (depth - depth.mean()) / depth.std()
    body_corrected = body.copy()
    body_corrected[(z_score < -2) | (z_score > 2)] = 0
    return body_corrected, cv2.bitwise_and(depth, depth, mask=body_corrected)

# --- Surface Fitting ---
def quad_surface(xy, a, b, c, d, e, f):
    x, y = xy
    return a + b*x + c*y + d*x**2 + e*y**2 + f*x*y

def correct_depth_curvature(depth_component, wound, body_corrected):
    margin = 40
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    wound_dilated = cv2.dilate(wound, kernel, iterations=margin)
    wound_eroded = cv2.erode(wound, kernel, iterations=margin)
    peri_lesion_mask = cv2.bitwise_xor(wound_dilated, wound_eroded)
    peri_lesion_mask = cv2.bitwise_and(peri_lesion_mask, peri_lesion_mask, mask=body_corrected)

    coords_y, coords_x = np.where(peri_lesion_mask > 0)
    
    if len(coords_x) == 0 or len(coords_y) == 0:
        raise ValueError("Peri-lesion mask is empty, skipping curve fitting.")
    
    z_vals = depth_component[coords_y, coords_x]
    params, _ = curve_fit(quad_surface, (coords_x, coords_y), z_vals)

    h, w = depth_component.shape
    grid_y, grid_x = np.mgrid[0:h, 0:w]
    fitted_surface = quad_surface((grid_x, grid_y), *params)

    depth_corrected = depth_component.astype(np.float32) - fitted_surface
    depth_corrected[body_corrected == 0] = 0
    return depth_corrected

# --- Rectification Tools ---
def get_pixel_boundaries(img: np.ndarray, mask: np.ndarray) -> np.ndarray:
    cnt, _ = cv2.findContours(image=mask, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
    cnt = max(cnt, key=len)
    cnt = np.squeeze(cnt)
    pixels = np.swapaxes(img, axis1=0, axis2=1)
    pixels = pixels[tuple(cnt.T)]
    return np.expand_dims(pixels, axis=1)

def rectify_boundary_mask(img: np.ndarray, mask: np.ndarray, iterations: int = 100):
    pxl1 = get_pixel_boundaries(img, mask)
    h, w, *_ = pxl1.shape
    dilations = [pxl1]

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    temp = mask.copy()
    for _ in range(iterations):
        temp = cv2.dilate(temp, kernel, iterations=1)
        pxl = get_pixel_boundaries(img, temp)
        pxl = cv2.resize(pxl, dsize=(w, h), interpolation=cv2.INTER_CUBIC)
        dilations.append(pxl)

    temp = mask.copy()
    erosions = []
    for _ in range(iterations):
        temp = cv2.erode(temp, kernel, iterations=1)
        ncomp, _ = cv2.connectedComponents(temp)
        if ncomp > 2:
            break
        pxl = get_pixel_boundaries(img, temp)
        pxl = cv2.resize(pxl, dsize=(w, h), interpolation=cv2.INTER_CUBIC)
        erosions.append(pxl)

    rectify_img = np.hstack(erosions[::-1] + dilations)
    return rectify_img.transpose(1, 0) if img.ndim == 2 else rectify_img.transpose(1, 0, 2)

# --- Feature Extraction ---
def extract_profile_features(profile):
    mean_val = np.mean(profile)
    std_val = np.std(profile)
    min_val = np.min(profile)
    max_val = np.max(profile)
    range_val = max_val - min_val
    skewness = skew(profile)
    kurt = kurtosis(profile)

    slope = np.gradient(profile)
    max_slope = np.max(np.abs(slope))
    positions = np.arange(len(profile))
    center_of_mass = np.sum(positions * profile) / np.sum(profile)
    area = simpson(profile)

    fft_vals = np.abs(fft(profile))
    fft_features = fft_vals[1:11]  # skip DC

    return [
        mean_val, std_val, min_val, max_val, range_val,
        skewness, kurt, max_slope, center_of_mass, area
    ] + list(fft_features)

# --- Main Pipeline Entry Point ---
def process_single_wound(name, base_path="../data"):
    img, depth, wound, body = load_inputs(name, base_path)
    body_corrected, depth_component = filter_body_with_zscore(depth, body)
    depth_corrected = correct_depth_curvature(depth_component, wound, body_corrected)
    blurred_depth = cv2.GaussianBlur(depth_corrected.astype(np.float32), (5, 5), sigmaX=1)
    strip = rectify_boundary_mask(np.where(blurred_depth != 0, blurred_depth, np.nan), wound, iterations=100)
    mean_profile = np.nanmean(strip, axis=0)
    mean_profile_smooth = gaussian_filter1d(mean_profile, sigma=2)
    return extract_profile_features(mean_profile_smooth)

## Extracting Image Names 

In [2]:
import pandas as pd
import os

image_ids = [
    os.path.splitext(f)[0]
    for f in os.listdir("../data/wound_masks")
    if f.endswith(".png")
]
df = pd.DataFrame(image_ids, columns=["image_id"])
df.to_csv("image_index.csv", index=False)

## Extracting Features Vectors from the Sample

In [12]:
import numpy as np
df = pd.read_csv("image_index.csv")
features_all = []
valid_ids = []

for name in df["image_id"]:
    try:
        features = process_single_wound(name, base_path="../data")

        # Skip feature vectors that are all NaNs
        if np.isnan(features).all():
            print(f"Skipped {name} — all features are NaN.")
            continue

        features_all.append(features)
        valid_ids.append(name)

    except Exception as e:
        print(f"Error with {name}: {e}")


feature_names = [
    "mean", "std", "min", "max", "range", "skewness", "kurtosis",
    "max_slope", "center_of_mass", "area"
] + [f"fft_{i}" for i in range(1, 11)]

features_df = pd.DataFrame(features_all, columns=feature_names)
features_df.insert(0, "image_id", valid_ids)

# Save only valid rows
features_df.to_csv("wound_features_clean.csv", index=False)

Error with -LzGpa0VPSMJr2F9rDCN.-LzGpeFMtDMDIZw91eP3.-MIYUwBw1GSWCJ3DThGw: Peri-lesion mask is empty, skipping curve fitting.
Error with -MyBpdFwPklsT0NHzF24.-MyBqvYdSk3E-aN9gMQx.-NOU0MMYnBuSpskbS6Pg: Peri-lesion mask is empty, skipping curve fitting.
Error with -Ll6cs_2_PISse2tMbIY.-Ll6d2JxKsxCBH7cEtzy.-Myl42G7DWh5zMANqQo1: Peri-lesion mask is empty, skipping curve fitting.
Error with -Lht2eTXnmfpJmP58xnY.-Lht5kX1F8HLIO_XlCaJ.-LsGoY6abAp6UFtnI0KN: Peri-lesion mask is empty, skipping curve fitting.
Error with -Moc0BVPVOr60FcVWR7b.-Moc0LZ_JePj2ozbLWp8.-MxYqSV4GrJO6D0mz4ZC: Peri-lesion mask is empty, skipping curve fitting.
Error with -M107tRXHadUY985DOxK.-M108LEQ4GxeLwj2uL2i.-M7M3VWpgYp5yB1Enkya: Peri-lesion mask is empty, skipping curve fitting.
Error with -LkXdHJF83_HuTFp1rKX.-LkXdNOQzzEmDdZ3ixjQ.-N6qHYv-gkVTLlzgnysF: Peri-lesion mask is empty, skipping curve fitting.
Error with -MxYN_-GGjd3aJ0ZnbsD.-N-2Ps6d4kaBcBQOWZks.-N-2R6NY12NKczegvaEd: Peri-lesion mask is empty, skipping curve f

/var/folders/v4/vp4_jyrx03x_4_00k4fv8vyh0000gn/T/ipykernel_13712/62894081.py:141: RuntimeWarning: Mean of empty slice
  mean_profile = np.nanmean(strip, axis=0)


Skipped -MqxMJ3lfkYcQvx3vuB6.-MqxMOtop_fRx4yVmH2N.-MqxMTPW2u2PT2G16KlC — all features are NaN.
Error with -MAV3HS34pHY2i1-YSV_.-MAV3WJjcTNFpt1CbDIQ.-MHAwUrYdUzySPDt6YgO: Peri-lesion mask is empty, skipping curve fitting.
Error with -LkXdHJF83_HuTFp1rKX.-LkXdNOQzzEmDdZ3ixjQ.-MeYmaqQtxLaowsez68l: Peri-lesion mask is empty, skipping curve fitting.
Error with -MuymHVjAwuWar9bxOtO.-Muymz6J-KCn7Sz4EoLL.-NNetx3PFxfvJkO2HH88: Peri-lesion mask is empty, skipping curve fitting.
Error with -N2Ajl6_8RGCd6aajyRP.-N34BMyhxA1JDev2cXQF.-N5tFWAGbkXq70aeBVsW: Peri-lesion mask is empty, skipping curve fitting.
Skipped -M2gsMSjgBDeZhPiRGCs.-M2gsd9LQMlBwPrVPX3L.-M4s-gEHK92n_03cnL7L — all features are NaN.
Error with -LpXjJ_FzS9HCyLgvzTR.-LrmKkSh8e0qj9ZEdNm9.-LrmLYKkcWFovgNTorui: Peri-lesion mask is empty, skipping curve fitting.
Error with -Lhi_hQB4d9hP668bC18.-Lhi_pFqfsm7Fzt3bPH5.-NQOxLWj4715TOSAmJEj: max() arg is an empty sequence
Error with -NHTSIH6WGEuYYUbVYA2.-NHTSRVS7egqeBUpedyd.-NPvdkdpm1w_sw9O8ZX6: